In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import math

%matplotlib inline

plt.style.use('ggplot')
font = {'weight': 'bold',
        'size':   16}
plt.rc('font', **font)

In [86]:
# Make the geographical map:
filepath = '/Users/will/Desktop/Portland/calls_for_service/CAD-combined.csv'
df = pd.read_csv(filepath)

In [87]:
# remove second column if needed
df.drop('Unnamed: 0', axis=1, inplace=True)

In [88]:
# import make_datetime_column function from the src file:
%run /Users/will/dsi/PortlandPD/src/make_datetime_column.py

col_with_dates = 'ReportMonthYear'
df = make_datetime_column(df, col_with_dates)
df

,Address,CallNumber,FinalCallCategory,FinalCallGroup,Neighborhood,OpenDataLat,OpenDataLon,OpenDataX,OpenDataY,Priority,ReportMonthYear,ResponseTime_sec,TimeInQueue_sec,TravelTime_sec
0,NaN,100895132,NaN,NaN,Cathedral Park,NaN,NaN,NaN,NaN,High,2012-02-01,267,51,216
1,NaN,105350927,NaN,NaN,Powellhurst-Gilbert,NaN,NaN,NaN,NaN,Low,2012-03-01,"5,960","5,446",514
2,NaN,105707241,NaN,NaN,Roseway,NaN,NaN,NaN,NaN,High,2012-03-01,452,26,426
3,NaN,108493427,NaN,NaN,Lloyd,NaN,NaN,NaN,NaN,Low,2012-03-01,"2,400",209,"2,191"
4,NaN,125521122,NaN,NaN,Lloyd,NaN,NaN,NaN,NaN,High,2012-06-01,388,203,185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1781768,WB TILIKUM CROSSING BRG AT / MIDSPAN,2704855833,Assist,Assist,NaN,45.504962,-122.666907,7646617.0,677646.0,High,2019-04-01,263,55,208
1781769,WILDWOOD TRAIL AT / BIRCH TRAIL,2689569532,Assist,Assist,Forest Park,45.535284,-122.730493,7630626.0,689141.0,High,2019-03-01,753,28,725
1781770,WILDWOOD TRAIL AT / LOWER MACLEAY TRAIL,2776332122,Assist,Assist,Forest Park,45.528171,-122.725038,7631952.0,686510.0,High,2019-10-01,"1,172",65,"1,107"
1781771,WILDWOOD TRAIL AT / MP3,2692494414,Suspicious,Disorder,Arlington Heights,45.519725,-122.717564,7633782.0,683379.0,Medium,2019-03-01,"3,359","3,359",0


In [90]:
# Sort values by the reporting month:
df = df.sort_values(by='ReportMonthYear')

In [91]:
vc = df[df['FinalCallCategory'] == 'Vice']

In [92]:
# Group the calls by various dates, pre and post weed legalization:
# Weed legal to USE: 2015-07-01
vc_preweeduselegal = vc[vc['ReportMonthYear'] < '2015-07-01']
df_preweeduselegal = df[df['ReportMonthYear'] < '2015-07-01']
vc_postweeduselegal = vc[vc['ReportMonthYear'] >= '2015-07-01']
df_postweeduselegal = df[df['ReportMonthYear'] >= '2015-07-01']

# Weed is legal to SELL: 2015-10-01
vc_preweedstorelegal = vc[vc['ReportMonthYear'] < '2015-10-01']
df_preweedstorelegal = df[df['ReportMonthYear'] < '2015-10-01']
vc_postweedstorelegal = vc[vc['ReportMonthYear'] >= '2015-10-01']
df_postweedstorelegal = df[df['ReportMonthYear'] >= '2015-10-01']

In [93]:
# Portland location: 45.5051° N, 122.6750° W
# Create a blank map:
portland_coordinates = [45.5236, -122.6750]
map = folium.Map(location=portland_coordinates, zoom_start=11)



In [94]:
from folium.plugins import MarkerCluster
mc = MarkerCluster().add_to(map)


# for speed purposes
MAX_RECORDS = 100
  
 
# add a marker for every record in the filtered data, use a clustered view
for each in vc[0:MAX_RECORDS].iterrows():
    if math.isnan(each[1]['OpenDataLat']) or math.isnan(each[1]['OpenDataLon']):
        continue
    else:
        folium.Marker(location = [each[1]['OpenDataLat'],each[1]['OpenDataLon']],
                     clustered_marker = True).add_to(mc)

# Can be used to set pop-up values for the markers:        
# popup=each[1]['Category'] + ": " + each[1]['Descript']).add_to(mc)

display(map)

In [95]:
from folium.plugins import MarkerCluster
mc = MarkerCluster()


# for speed purposes
MAX_RECORDS = 100

# add a marker for every record in the filtered data, use a clustered view
for each in vc[0:MAX_RECORDS].iterrows():
    if math.isnan(each[1]['OpenDataLat']) or math.isnan(each[1]['OpenDataLon']):
        continue
    else:
        mc.add_child(folium.Marker(location = [each[1]['OpenDataLat'],each[1]['OpenDataLon']]))

# Can be used to set pop-up values for the markers:        
# popup=each[1]['Category'] + ": " + each[1]['Descript']).add_to(mc)

map.add_child(mc)
map